In [1]:
import tensorflow as tf
import numpy as np
import os
import scipy.io
import glob
import scipy.misc as misc
import warnings
from distutils.version import LooseVersion
import helper
import project_tests as tests

In [2]:
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.8.0


In [3]:
# Check for a GPU
if not tf.test.gpu_device_name():
  warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
  print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

/home/gother/.local/lib/python3.5/site-packages/ipykernel_launcher.py:3: UserWarning: No GPU found. Please use a GPU to train your neural network.
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
DATA_DIRECTORY = './data'
RUNS_DIRECTORY = './runs'
TRAINING_DATA_DIRECTORY ='./data/data_liquid/training'
NUMBER_OF_IMAGES = len(glob.glob('./data/data_liquid/testing/image_2/*.*'))
VGG_PATH = './data/vgg'

In [5]:
NUMBER_OF_CLASSES = 2
IMAGE_SHAPE = (480, 640)

EPOCHS = 20
BATCH_SIZE = 1

LEARNING_RATE = 0.0001
DROPOUT = 0.75

In [6]:
correct_label = tf.placeholder(tf.float32, [None, IMAGE_SHAPE[0], IMAGE_SHAPE[1], NUMBER_OF_CLASSES])
learning_rate = tf.placeholder(tf.float32)
keep_prob = tf.placeholder(tf.float32)

In [7]:
# Used for plotting to visualize if our training is going well given parameters
all_training_losses = []

In [8]:
def load_vgg(sess, vgg_path):
  """
  Load Pretrained VGG Model into TensorFlow.
  sess: TensorFlow Session
  vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
  return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3, layer4, layer7)
  """
  # load the model and weights
  model = tf.saved_model.loader.load(sess, ['vgg16'], vgg_path)

  # Get Tensors to be returned from graph
  graph = tf.get_default_graph()
  image_input = graph.get_tensor_by_name('image_input:0')
  keep_prob = graph.get_tensor_by_name('keep_prob:0')
  layer3 = graph.get_tensor_by_name('layer3_out:0')
  layer4 = graph.get_tensor_by_name('layer4_out:0')
  layer7 = graph.get_tensor_by_name('layer7_out:0')

  return image_input, keep_prob, layer3, layer4, layer7

In [9]:
def conv_1x1(layer, layer_name):
  """ Return the output of a 1x1 convolution of a layer """
  return tf.layers.conv2d(inputs = layer,
                          filters =  NUMBER_OF_CLASSES,
                          kernel_size = (1, 1),
                          strides = (1, 1),
                          name = layer_name)

In [10]:
def upsample(layer, k, s, layer_name):
  """ Return the output of transpose convolution given kernel_size k and strides s """
  # See: http://deeplearning.net/software/theano/tutorial/conv_arithmetic.html#transposed-convolution-arithmetic
  return tf.layers.conv2d_transpose(inputs = layer,
                                    filters = NUMBER_OF_CLASSES,
                                    kernel_size = (k, k),
                                    strides = (s, s),
                                    padding = 'same',
                                    name = layer_name)

In [11]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes = NUMBER_OF_CLASSES):
  """
  Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
  vgg_layerX_out: TF Tensor for VGG Layer X output
  num_classes: Number of classes to classify
  return: The Tensor for the last layer of output
  """

  # Use a shorter variable name for simplicity
  layer3, layer4, layer7 = vgg_layer3_out, vgg_layer4_out, vgg_layer7_out

  # Apply a 1x1 convolution to encoder layers
  layer3x = conv_1x1(layer = layer3, layer_name = "layer3conv1x1")
  layer4x = conv_1x1(layer = layer4, layer_name = "layer4conv1x1")
  layer7x = conv_1x1(layer = layer7, layer_name = "layer7conv1x1")
 
  # Add decoder layers to the network with skip connections and upsampling
  # Note: the kernel size and strides are the same as the example in Udacity Lectures
  #       Semantic Segmentation Scene Understanding Lesson 10-9: FCN-8 - Decoder
  decoderlayer1 = upsample(layer = layer7x, k = 4, s = 2, layer_name = "decoderlayer1")
  decoderlayer2 = tf.add(decoderlayer1, layer4x, name = "decoderlayer2")
  decoderlayer3 = upsample(layer = decoderlayer2, k = 4, s = 2, layer_name = "decoderlayer3")
  decoderlayer4 = tf.add(decoderlayer3, layer3x, name = "decoderlayer4")
  decoderlayer_output = upsample(layer = decoderlayer4, k = 16, s = 8, layer_name = "decoderlayer_output")

  return decoderlayer_output

In [12]:
def layers_verbose(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes = NUMBER_OF_CLASSES):

  # Use a shorter variable name for simplicity
  layer3, layer4, layer7 = vgg_layer3_out, vgg_layer4_out, vgg_layer7_out

  # Apply a 1x1 convolution to encoder layers
  layer3x = conv_1x1(layer = layer3, layer_name = "layer3conv1x1")
  layer4x = conv_1x1(layer = layer4, layer_name = "layer4conv1x1")
  layer7x = conv_1x1(layer = layer7, layer_name = "layer7conv1x1")
 
  decoderlayer1 = upsample(layer = layer7x, k = 4, s = 2, layer_name = "decoderlayer1")
  decoderlayer2 = tf.add(decoderlayer1, layer4x, name = "decoderlayer2")
  decoderlayer3 = upsample(layer = decoderlayer2, k = 4, s = 2, layer_name = "decoderlayer3")
  decoderlayer4 = tf.add(decoderlayer3, layer3x, name = "decoderlayer4")
  decoderlayer_output = upsample(layer = decoderlayer4, k = 16, s = 8, layer_name = "decoderlayer_output")

  return layer3, layer4, layer7, layer3x, layer4x, layer7x, \
         decoderlayer1, decoderlayer2, decoderlayer3, decoderlayer4, decoderlayer_output


In [13]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes = NUMBER_OF_CLASSES):
  """
  Build the TensorFLow loss and optimizer operations.
  nn_last_layer: TF Tensor of the last layer in the neural network
  correct_label: TF Placeholder for the correct label image
  learning_rate: TF Placeholder for the learning rate
  num_classes: Number of classes to classify
  return: Tuple of (logits, train_op, cross_entropy_loss)
  """
  # reshape 4D tensors to 2D
  # Each row represents a pixel, each column a class
  logits = tf.reshape(nn_last_layer, (-1, num_classes))
  class_labels = tf.reshape(correct_label, (-1, num_classes))

  # The cross_entropy_loss is the cost which we are trying to minimize to yield higher accuracy
  cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = class_labels)
  cross_entropy_loss = tf.reduce_mean(cross_entropy)

  # The model implements this operation to find the weights/parameters that would yield correct pixel labels
  train_op = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy_loss)

  return logits, train_op, cross_entropy_loss

In [14]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op,
             cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
  """
  Train neural network and print out the loss during training.
  sess: TF Session
  epochs: Number of epochs
  batch_size: Batch size
  get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
  train_op: TF Operation to train the neural network
  cross_entropy_loss: TF Tensor for the amount of loss
  input_image: TF Placeholder for input images
  correct_label: TF Placeholder for label images
  keep_prob: TF Placeholder for dropout keep probability
  learning_rate: TF Placeholder for learning rate
  """

  for epoch in range(EPOCHS):
    
    losses, i = [], 0
    
    for images, labels in get_batches_fn(BATCH_SIZE):
        
      i += 1
    
      feed = { input_image: images,
               correct_label: labels,
               keep_prob: DROPOUT,
               learning_rate: LEARNING_RATE }
        
      _, partial_loss = sess.run([train_op, cross_entropy_loss], feed_dict = feed)
      
      print("---> iteration: ", i, " partial loss:", partial_loss)
      losses.append(partial_loss)
          
    training_loss = sum(losses) / len(losses)
    all_training_losses.append(training_loss)
    
    print("------------------")
    print("epoch: ", epoch + 1, " of ", EPOCHS, "training loss: ", training_loss)
    print("------------------")

In [15]:
def run_tests():
  tests.test_layers(layers)
  tests.test_optimize(optimize)
  tests.test_for_kitti_dataset(DATA_DIRECTORY)
  tests.test_train_nn(train_nn)

In [16]:
def run():
  print("NUMBER OF IMAGES:", NUMBER_OF_IMAGES)

  # download vgg model
  helper.maybe_download_pretrained_vgg(DATA_DIRECTORY)

  # A function to get batches
  get_batches_fn = helper.gen_batch_function(TRAINING_DATA_DIRECTORY, IMAGE_SHAPE)
  
  with tf.Session() as session:
        
    # Returns the three layers, keep probability and input layer from the vgg architecture
    image_input, keep_prob, layer3, layer4, layer7 = load_vgg(session, VGG_PATH)

    # The resulting network architecture, adding a decoder on top of the given vgg model
    model_output = layers(layer3, layer4, layer7, NUMBER_OF_CLASSES)

    # Returns the output logits, training operation and cost operation to be used
    # For the logits: each row represents a pixel, each column a class
    # training operation is what is used to get the right parameters to the model to correctly label the pixels
    # the cross entropy loss is the cost which we are minimizing, lower cost should yield higher accuracy
    logits, train_op, cross_entropy_loss = optimize(model_output, correct_label, learning_rate, NUMBER_OF_CLASSES)
    
    # Initilize all variables
    session.run([tf.global_variables_initializer(), tf.local_variables_initializer()])

    # train the neural network
    train_nn(session, EPOCHS, BATCH_SIZE, get_batches_fn, 
             train_op, cross_entropy_loss, image_input,
             correct_label, keep_prob, learning_rate)

    # Save inference data
    helper.save_inference_samples(RUNS_DIRECTORY, DATA_DIRECTORY, session, IMAGE_SHAPE, logits, keep_prob, image_input)
    
    # Save model
    saver.save(session, "fcn_liquid_model", global_step = NUMBER_OF_IMAGES * 2)

In [17]:
def network_shapes():
  with tf.Session() as sess:
    
    x = np.random.randn(1, 160, 576, 3)
    
    image_input, keep_prob, layer3, layer4, layer7 = load_vgg(sess, VGG_PATH)
 
    op = layers_verbose(layer3, layer4, layer7, NUMBER_OF_CLASSES)
  
    sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])

    l3, l4, l7, l3x, l4x, l7x, d1, s2, d3, s4, d5 = sess.run(op, feed_dict = {image_input: x, keep_prob: 1.0})

    print("------------------")
    print("shapes of layers:") 
    print("------------------")

    print("layer3 -->", l3.shape)
    print("layer4 -->", l4.shape)
    print("layer7 -->", l7.shape)
    print("layer3 conv1x1 -->", l3x.shape)
    print("layer4 conv1x1 -->", l4x.shape)
    print("layer7 conv1x1-->", l7x.shape)
    print("decoderlayer1 transpose: layer7 k = 4 s = 2 -->", d1.shape)
    print("decoderlayer2 skip: decoderlayer1 and layer4conv1x1 -->", s2.shape)
    print("decoderlayer3 transpose: decoderlayer2 k = 4 s = 2 -->", d3.shape)
    print("decoderlayer4 skip: decoderlayer3 and layer3conv1x1 -->", s4.shape)
    print("decoderlayer5 transpose: decoderlayer4 k = 16 s = 8 -->", d5.shape)

In [18]:
# Train the network
run()

NUMBER OF IMAGES: 508
INFO:tensorflow:Restoring parameters from b'./data/vgg/variables/variables'
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

---> iteration:  1  partial loss: 71.16544
---> iteration:  2  partial loss: 36.057938
---> iteration:  3  partial loss: 38.122154
---> iteration:  4  partial loss: 30.437214
---> iteration:  5  partial loss: 23.65168
---> iteration:  6  partial loss: 20.518719
---> iteration:  7  partial loss: 19.932825
---> iteration:  8  partial loss: 17.433596
---> iteration:  9  partial loss: 14.972803
---> iteration:  10  partial loss: 12.666071
---> iteration:  11  partial loss: 11.405317
---> iteration:  12  partial loss: 10.84542
---> iteration:  13  partial loss: 9.779578
---> iteration:  14  partial loss: 8.979368
---> iteration:  15  partial loss: 8.073193
---> iteration:  16  partial loss: 7.2187533
---> 

---> iteration:  171  partial loss: 0.77989805
---> iteration:  172  partial loss: 0.78173476
---> iteration:  173  partial loss: 0.7748011
---> iteration:  174  partial loss: 0.7744653
---> iteration:  175  partial loss: 0.7696465
---> iteration:  176  partial loss: 0.7709002
---> iteration:  177  partial loss: 0.7744101
---> iteration:  178  partial loss: 0.76812136
---> iteration:  179  partial loss: 0.7759013
---> iteration:  180  partial loss: 0.7779029
---> iteration:  181  partial loss: 0.7725641
---> iteration:  182  partial loss: 0.76680297
---> iteration:  183  partial loss: 0.7587916
---> iteration:  184  partial loss: 0.76725596
---> iteration:  185  partial loss: 0.76287943
---> iteration:  186  partial loss: 0.7625422
---> iteration:  187  partial loss: 0.7626608
---> iteration:  188  partial loss: 0.7583795
---> iteration:  189  partial loss: 0.76009625
---> iteration:  190  partial loss: 0.75235885
---> iteration:  191  partial loss: 0.7520436
---> iteration:  192  part

---> iteration:  348  partial loss: 0.6871048
---> iteration:  349  partial loss: 0.68403774
---> iteration:  350  partial loss: 0.6828384
---> iteration:  351  partial loss: 0.6850821
---> iteration:  352  partial loss: 0.6844082
---> iteration:  353  partial loss: 0.6849231
---> iteration:  354  partial loss: 0.67935336
---> iteration:  355  partial loss: 0.6842444
---> iteration:  356  partial loss: 0.6808501
---> iteration:  357  partial loss: 0.67880684
---> iteration:  358  partial loss: 0.68338287
---> iteration:  359  partial loss: 0.68040675
---> iteration:  360  partial loss: 0.6816166
---> iteration:  361  partial loss: 0.67812926
---> iteration:  362  partial loss: 0.681818
---> iteration:  363  partial loss: 0.67348814
---> iteration:  364  partial loss: 0.68023074
---> iteration:  365  partial loss: 0.67752093
---> iteration:  366  partial loss: 0.68126637
---> iteration:  367  partial loss: 0.67113674
---> iteration:  368  partial loss: 0.68051064
---> iteration:  369  p

---> iteration:  16  partial loss: 0.6447637
---> iteration:  17  partial loss: 0.6444028
---> iteration:  18  partial loss: 0.6429059
---> iteration:  19  partial loss: 0.643608
---> iteration:  20  partial loss: 0.6433831
---> iteration:  21  partial loss: 0.6345465
---> iteration:  22  partial loss: 0.6460138
---> iteration:  23  partial loss: 0.63430583
---> iteration:  24  partial loss: 0.6328423
---> iteration:  25  partial loss: 0.6416139
---> iteration:  26  partial loss: 0.6412725
---> iteration:  27  partial loss: 0.6412141
---> iteration:  28  partial loss: 0.6343614
---> iteration:  29  partial loss: 0.6407606
---> iteration:  30  partial loss: 0.6321258
---> iteration:  31  partial loss: 0.63230354
---> iteration:  32  partial loss: 0.6415098
---> iteration:  33  partial loss: 0.6319293
---> iteration:  34  partial loss: 0.6397
---> iteration:  35  partial loss: 0.6394265
---> iteration:  36  partial loss: 0.629421
---> iteration:  37  partial loss: 0.63603985
---> iterati

---> iteration:  195  partial loss: 0.5977001
---> iteration:  196  partial loss: 0.57722086
---> iteration:  197  partial loss: 0.592444
---> iteration:  198  partial loss: 0.5911607
---> iteration:  199  partial loss: 0.5816809
---> iteration:  200  partial loss: 0.58976895
---> iteration:  201  partial loss: 0.5743981
---> iteration:  202  partial loss: 0.5762333
---> iteration:  203  partial loss: 0.59357125
---> iteration:  204  partial loss: 0.58898604
---> iteration:  205  partial loss: 0.58993804
---> iteration:  206  partial loss: 0.5905885
---> iteration:  207  partial loss: 0.5873608
---> iteration:  208  partial loss: 0.57189023
---> iteration:  209  partial loss: 0.5925617
---> iteration:  210  partial loss: 0.5914613
---> iteration:  211  partial loss: 0.58049226
---> iteration:  212  partial loss: 0.5858645
---> iteration:  213  partial loss: 0.5869355
---> iteration:  214  partial loss: 0.5746612
---> iteration:  215  partial loss: 0.5864
---> iteration:  216  partial l

...

---> iteration:  244  partial loss: 3.99823e-07
---> iteration:  245  partial loss: 0.0059419153
---> iteration:  246  partial loss: 0.0058410903
---> iteration:  247  partial loss: 0.004397154
---> iteration:  248  partial loss: 0.0048106513
---> iteration:  249  partial loss: 0.004281967
---> iteration:  250  partial loss: 5.078985e-06
---> iteration:  251  partial loss: 0.003998724
---> iteration:  252  partial loss: 0.0057221795
---> iteration:  253  partial loss: 9.769857e-07
---> iteration:  254  partial loss: 0.013123914
---> iteration:  255  partial loss: 2.3158477e-06
---> iteration:  256  partial loss: 1.4780901e-06
---> iteration:  257  partial loss: 0.005729834
---> iteration:  258  partial loss: 0.004639952
---> iteration:  259  partial loss: 0.0053598736
---> iteration:  260  partial loss: 0.004356762
---> iteration:  261  partial loss: 0.011131517
---> iteration:  262  partial loss: 6.584011e-07
---> iteration:  263  partial loss: 0.005530004
---> iteration:  264  partia

---> iteration:  412  partial loss: 6.0786135e-07
---> iteration:  413  partial loss: 0.010721687
---> iteration:  414  partial loss: 7.308632e-07
---> iteration:  415  partial loss: 9.4498535e-07
---> iteration:  416  partial loss: 2.3443051e-06
---> iteration:  417  partial loss: 1.312033e-06
---> iteration:  418  partial loss: 0.0059335944
---> iteration:  419  partial loss: 7.607268e-07
---> iteration:  420  partial loss: 0.0052972585
---> iteration:  421  partial loss: 0.009013746
---> iteration:  422  partial loss: 3.9057002e-07
---> iteration:  423  partial loss: 0.009124923
---> iteration:  424  partial loss: 0.011815501
---> iteration:  425  partial loss: 0.005036762
---> iteration:  426  partial loss: 5.349763e-07
---> iteration:  427  partial loss: 0.002869172
---> iteration:  428  partial loss: 0.0055256216
---> iteration:  429  partial loss: 0.0065783546
---> iteration:  430  partial loss: 0.002923623
---> iteration:  431  partial loss: 0.004128937
---> iteration:  432  pa

---> iteration:  72  partial loss: 0.006543568
---> iteration:  73  partial loss: 0.0041436413
---> iteration:  74  partial loss: 0.0067959307
---> iteration:  75  partial loss: 0.010521759
---> iteration:  76  partial loss: 0.0074965516
---> iteration:  77  partial loss: 0.016319687
---> iteration:  78  partial loss: 0.011079475
---> iteration:  79  partial loss: 0.005070844
---> iteration:  80  partial loss: 0.0068899975
---> iteration:  81  partial loss: 0.0055115167
---> iteration:  82  partial loss: 0.004714442
---> iteration:  83  partial loss: 3.999107e-05
---> iteration:  84  partial loss: 0.0038418032
---> iteration:  85  partial loss: 2.0190422e-05
---> iteration:  86  partial loss: 0.005037473
---> iteration:  87  partial loss: 0.0013408104
---> iteration:  88  partial loss: 0.0034343386
---> iteration:  89  partial loss: 0.015360575
---> iteration:  90  partial loss: 0.003491346
---> iteration:  91  partial loss: 1.971575e-06
---> iteration:  92  partial loss: 5.115879e-06


---> iteration:  241  partial loss: 3.1587395e-05
---> iteration:  242  partial loss: 0.006726734
---> iteration:  243  partial loss: 0.0058445265
---> iteration:  244  partial loss: 2.756693e-06
---> iteration:  245  partial loss: 7.459401e-06
---> iteration:  246  partial loss: 0.012096837
---> iteration:  247  partial loss: 1.5465239e-05
---> iteration:  248  partial loss: 0.0042302925
---> iteration:  249  partial loss: 0.005472672
---> iteration:  250  partial loss: 0.0048953556
---> iteration:  251  partial loss: 0.0043540555
---> iteration:  252  partial loss: 0.005103228
---> iteration:  253  partial loss: 0.00015919126
---> iteration:  254  partial loss: 0.0035870813
---> iteration:  255  partial loss: 0.0049654236
---> iteration:  256  partial loss: 0.004443199
---> iteration:  257  partial loss: 5.1821135e-05
---> iteration:  258  partial loss: 1.0606475e-05
---> iteration:  259  partial loss: 0.015556164
---> iteration:  260  partial loss: 1.1319975e-05
---> iteration:  261

---> iteration:  409  partial loss: 0.0051539177
---> iteration:  410  partial loss: 5.044885e-07
---> iteration:  411  partial loss: 1.5022333e-07
---> iteration:  412  partial loss: 0.005604861
---> iteration:  413  partial loss: 0.004113749
---> iteration:  414  partial loss: 8.3065737e-07
---> iteration:  415  partial loss: 2.4107823e-07
---> iteration:  416  partial loss: 5.4643515e-08
---> iteration:  417  partial loss: 9.3562605e-08
---> iteration:  418  partial loss: 0.008649477
---> iteration:  419  partial loss: 0.010505464
---> iteration:  420  partial loss: 3.6932715e-07
---> iteration:  421  partial loss: 0.011398401
---> iteration:  422  partial loss: 0.0050237076
---> iteration:  423  partial loss: 0.003694303
---> iteration:  424  partial loss: 0.0049068113
---> iteration:  425  partial loss: 0.006809209
---> iteration:  426  partial loss: 0.004256371
---> iteration:  427  partial loss: 7.5622364e-07
---> iteration:  428  partial loss: 1.2164217e-07
---> iteration:  429

In [19]:
all_training_losses

[1.5580604142091405,
 0.4980435570569958,
 0.0807330846258505,
 0.058383451254055724,
 0.0338730459109748,
 0.019409216276629403,
 0.01512352138613676,
 0.012775516903146369,
 0.011273648990506953,
 0.0110301222393972,
 0.00931879426453691,
 0.008359735531617543,
 0.0216080526599173,
 0.008320302685196717,
 0.006564879546104519,
 0.005730839001792724,
 0.004888331277058539,
 0.004308389416073273,
 0.003978307920329495,
 0.004172437976839668]

In [ ]:
import os.path
from glob import glob
import re

data_folder = TRAINING_DATA_DIRECTORY
image_paths = glob(os.path.join(data_folder, 'image_2', '*.png'))
label_paths = {
           re.sub(r'ground_truth', 'data', os.path.basename(path)): path
            for path in glob(os.path.join(data_folder, 'gt_image_2', '*.png'))}
background_color = np.array([0, 0, 0, 0])

print (image_paths)
print ("@@@@@@@@@@@@@@@@@@@@@@@")
print (label_paths)

batch_size = BATCH_SIZE
image_shape = IMAGE_SHAPE

for batch_i in range(0, len(image_paths), batch_size):
    images = []
    gt_images = []
    for image_file in image_paths[batch_i:batch_i+batch_size]:
        gt_image_file = label_paths[os.path.basename(image_file)]

        image = scipy.misc.imresize(scipy.misc.imread(image_file), image_shape)
        gt_image = scipy.misc.imresize(scipy.misc.imread(gt_image_file), image_shape)
        gt_bg = np.all(gt_image == background_color, axis=2)
        gt_bg = gt_bg.reshape(*gt_bg.shape, 1)
        gt_image = np.concatenate((gt_bg, np.invert(gt_bg)), axis=2)

        images.append(image)
        gt_images.append(gt_image)

In [21]:
run_tests()

Tests Passed
Tests Passed
Tests Passed


In [22]:
helper.save_inference_samples(RUNS_DIRECTORY, DATA_DIRECTORY, session, IMAGE_SHAPE, logits, keep_prob, image_input)

NameError: name 'session' is not defined